In [143]:
from collections import OrderedDict, defaultdict

!pip3 install pickle5
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import pickle5 as pickle
import matplotlib.pyplot as plt
import matplotlib.style as style
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset

from google.colab import drive

drive.mount("/content/drive")
%cd '/content/drive/MyDrive/master/cs7643/project/'       

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/master/cs7643/project


In [144]:
class VanillaRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout, bidirectional):
        super(VanillaRNN, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(input_size, hidden_dim, num_layers=n_layers, batch_first=True, nonlinearity='relu', bidirectional=bidirectional, dropout=dropout)   
        multiplier = 2 if bidirectional else 1
        self.fc = nn.Sequential(
          nn.Linear(n_layers * hidden_dim * multiplier, output_size * 2),
          nn.BatchNorm1d(output_size * 2),
          nn.ReLU(),
          nn.Linear(output_size * 2, output_size),
      )
    
    def forward(self, x):
        N = len(x)
        _, hidden = self.rnn(x)
        hidden = hidden.permute(1,0,2).contiguous().view(N, -1)
        hidden = self.fc(hidden)
        return hidden

class VanillaRNNWithCNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout, bidirectional):
        super(VanillaRNNWithCNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.cnn_encoder = nn.Sequential(
            nn.Conv1d(input_size, 8, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm1d(8),
            nn.ReLU(inplace=True),
            nn.Conv1d(8, 16, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm1d(16),
            nn.ReLU(inplace=True),
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
        )
        self.rnn = nn.RNN(32, hidden_dim, num_layers=n_layers, batch_first=True, bidirectional=bidirectional, nonlinearity='relu', dropout=dropout)   
        multiplier = 2 if bidirectional else 1
        self.fc = nn.Sequential(
          nn.Linear(n_layers * hidden_dim * multiplier, output_size * 2),
          nn.BatchNorm1d(output_size * 2),
          nn.ReLU(),
          nn.Linear(output_size * 2, output_size),
      )
    
    def forward(self, x):
      N = len(x)
      out = x.permute(0, 2, 1)
      out = self.cnn_encoder(out)
      out = out.permute(0, 2, 1)
      _, hidden = self.rnn(out)
      hidden = hidden.permute(1,0,2).contiguous().view(N, -1)
      hidden = self.fc(hidden)
      return hidden

class VanillaLSTM(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout, bidirectional):
        super(VanillaLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(input_size, hidden_dim, num_layers=n_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)   
        multiplier = 2 if bidirectional else 1
        self.fc = nn.Sequential(
          nn.Linear(n_layers * hidden_dim * multiplier, output_size * 2),
          nn.BatchNorm1d(output_size * 2),
          nn.ReLU(),
          nn.Linear(output_size * 2, output_size),
      )
    
  def forward(self, x):
      N = len(x)
      _, (hidden, _) = self.rnn(x)
      hidden = hidden.permute(1,0,2).contiguous().view(N, -1)
      hidden = self.fc(hidden)
      return hidden

class VanillaLSTMWithCNN(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout, bidirectional):
      super(VanillaLSTMWithCNN, self).__init__()

      self.hidden_dim = hidden_dim
      self.n_layers = n_layers

      self.cnn_encoder = nn.Sequential(
          nn.Conv1d(input_size, 8, kernel_size=3, stride=1, padding=1, bias=True),
          nn.BatchNorm1d(8),
          nn.ReLU(inplace=True),
          nn.Conv1d(8, 16, kernel_size=3, stride=1, padding=1, bias=True),
          nn.BatchNorm1d(16),
          nn.ReLU(inplace=True),
          nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1, bias=True),
          nn.BatchNorm1d(32),
          nn.ReLU(inplace=True),
      )
      self.rnn = nn.LSTM(32, hidden_dim, num_layers=n_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)   
      multiplier = 2 if bidirectional else 1
      self.fc = nn.Sequential(
          nn.Linear(n_layers * hidden_dim * multiplier, output_size * 2),
          nn.BatchNorm1d(output_size * 2),
          nn.ReLU(),
          nn.Linear(output_size * 2, output_size),
      )
  def forward(self, x):
      N = len(x)
      out = x.permute(0, 2, 1)
      out = self.cnn_encoder(out)
      out = out.permute(0, 2, 1)
      _, (hidden, _) = self.rnn(out)
      hidden = hidden.permute(1,0,2).contiguous().view(N, -1)
      hidden = self.fc(hidden)
      return hidden

class VanillaGRU(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout, bidirectional):
        super(VanillaGRU, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.GRU(input_size, hidden_dim, num_layers=n_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)   
        multiplier = 2 if bidirectional else 1
        self.fc = nn.Sequential(
            nn.Linear(n_layers * hidden_dim * multiplier, output_size * 2),
            nn.BatchNorm1d(output_size * 2),
            nn.ReLU(),
            nn.Linear(output_size * 2, output_size),
        )
    
  def forward(self, x):
      N = len(x)
      _, hidden = self.rnn(x)
      hidden = hidden.permute(1,0,2).contiguous().view(N, -1)
      hidden = self.fc(hidden)
      return hidden

class VanillaGRUWithCNN(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, n_layers, dropout, bidirectional):
      super(VanillaGRUWithCNN, self).__init__()

      self.hidden_dim = hidden_dim
      self.n_layers = n_layers

      self.cnn_encoder = nn.Sequential(
          nn.Conv1d(input_size, 8, kernel_size=3, stride=1, padding=1, bias=True),
          nn.BatchNorm1d(8),
          nn.ReLU(inplace=True),
          nn.Conv1d(8, 16, kernel_size=3, stride=1, padding=1, bias=True),
          nn.BatchNorm1d(16),
          nn.ReLU(inplace=True),
          nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1, bias=True),
          nn.BatchNorm1d(32),
          nn.ReLU(inplace=True),
      )
      self.rnn = nn.GRU(32, hidden_dim, num_layers=n_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)   
      multiplier = 2 if bidirectional else 1
      self.fc = nn.Sequential(
          nn.Linear(n_layers * hidden_dim * multiplier, output_size * 2),
          nn.BatchNorm1d(output_size * 2),
          nn.ReLU(),
          nn.Linear(output_size * 2, output_size),
      )
  def forward(self, x):
      N = len(x)
      out = x.permute(0, 2, 1)
      out = self.cnn_encoder(out)
      out = out.permute(0, 2, 1)
      _, hidden = self.rnn(out)
      hidden = hidden.permute(1,0,2).contiguous().view(N, -1)
      hidden = self.fc(hidden)
      return hidden

In [145]:
# Util functions
def draw(drawing):
  for x,y in drawing:
    plt.plot(x, y, marker='.')
    plt.axis('off')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.show()

def one_hot(category):
    arr = np.zeros(len(categories))
    arr[category] = 1.0
    return arr

def train(model, X_train, y_train, epochs = 1, n_chunks = 1000, learning_rate = 0.003, weight_decay = 0, optimizer = 'SGD'):
  print("Training model with epochs = {epochs}, learning rate = {lr}\n".format(epochs = epochs, lr = learning_rate))

  criterion = nn.CrossEntropyLoss()

  if optimizer == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  losses = []
  for epoch in range(epochs):
    running_loss = 0
    images = torch.chunk(X_train, n_chunks)
    labels = torch.chunk(y_train, n_chunks)

    N = len(images)

    for i in range(N):
      optimizer.zero_grad()
      output = model.forward(images[i])
      loss = criterion(output, labels[i])
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
    losses.append(running_loss)
    print("Epoch: {}/{}... ".format(epoch + 1, epochs), "Loss: {:.4f}".format(running_loss))
  return losses
  
def get_prediction_probability_distribution(model, input):
  with torch.no_grad():
    logits = model.forward(input)
  return torch.nn.functional.softmax(logits, dim=1)

def get_labels(model, input):
  probability_distribution = get_prediction_probability_distribution(model, input)
  pred_np = probability_distribution.numpy()
  N = len(pred_np)
  pred_values = np.amax(pred_np, axis=1, keepdims=True)
  pred_labels = np.array([np.where(pred_np[i, :] == pred_values[i, :])[0] for i in range(pred_np.shape[0])])
  pred_labels = pred_labels.reshape(N, 1)
  return pred_labels

def evaluate_model(model, train, y_train, test, y_test):
  train_pred_labels = get_labels(model, train)
  test_pred_labels = get_labels(model, test)
  y_train_label = np.argmax(y_train,axis=1)
  y_test_label = np.argmax(y_test,axis=1)
  accuracy_train = accuracy_score(y_train_label, train_pred_labels)
  accuracy_test = accuracy_score(y_test_label, test_pred_labels)

  print("Accuracy score for train set is {} \n".format(accuracy_train))
  print("Accuracy score for test set is {} \n".format(accuracy_test))

  return accuracy_train, accuracy_test

In [146]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [147]:
categories = ['flower', 'apple', 'baseball', 'baskebtall', 'bird', 'book', 'bus', 'car', 'cat', 'dog']
label_dict = {0:'flower', 1:'apple', 2:'baseball', 3:'baskebtall', 4:'bird',
                      5:'book',6:'bus', 7:'car', 8:'cat', 9:'dog'}

In [148]:
# Data Hyperparameters
test_set_split = 0.2
MAX_SEQUENCE_LENGTH = 50
EOS_TOKEN = None
SEQUENCE_DIMENSION = 3 # [x, y, is_stroke_start]

In [149]:
class RNNDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, parsed_data, transform=None):
        """
        Args:
            parsed_data (string): Path to the parsed_data.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        with open(parsed_data, "rb") as fh:
          self.file_pkg = pickle.load(fh)
        # self.file_pkg = pd.read_pickle(parsed_data)
        self.transform = transform

    def __len__(self):
        return len(self.file_pkg)

    def __getitem__(self, idx):
        return self.file_pkg.iloc[idx].to_dict()

def collate_fn(batch):
  N = len(batch)
  for i in range(N):
    seq = batch[i]['drawing']
    category = batch[i]['class']
    input = np.zeros((MAX_SEQUENCE_LENGTH, SEQUENCE_DIMENSION), np.float32)
    input[0:len(seq)] = seq[: MAX_SEQUENCE_LENGTH]
    batch[i]['drawing'] = input
  return batch

# Returns a DataLoader
# It already shuffles data as per sampler option
def read_rnn_data(batch_size=4, num_workers=0):
	transformed_dataset = RNNDataset('./rnn_parsed_data.pkl')
	return DataLoader(transformed_dataset, batch_size, num_workers, collate_fn=collate_fn)

In [150]:
# Load data for each category
classes = defaultdict(lambda: [])

# read data
ENABLE_MULTI_BATCH = False
READ_BATCH_SIZE = 50000
loader = read_rnn_data(batch_size=READ_BATCH_SIZE, num_workers=10)
for i_batch, sample_batched in enumerate(loader):
  N = len(sample_batched)
  for i in range(N):
    seq = sample_batched[i]['drawing']
    category = sample_batched[i]['class']
    classes[category].append(seq)
  if not ENABLE_MULTI_BATCH:
    break

# for category in categories:
#     # data = pd.read_csv("../input/train_simplified/" + category + ".csv")
#     data = [[[1,1,0],[2,2,0],[3,3,1],[4,4,1],[5,5,1]], [[1,1,0],[2,2,0],[3,3,1],[4,4,1],[5,5,1]]] # dummy
#     N = len(data)
#     input = np.zeros((N, MAX_SEQUENCE_LENGTH, SEQUENCE_DIMENSION), np.float32)
#     for seq in data:
#       length, _ = np.shape(seq)
      
#     for i in range(N):
#       seq = data[i]
#       input[i, 0:len(seq)] = seq[: MAX_SEQUENCE_LENGTH]
#     classes[category] = input

In [151]:
# Process input output pairs
X = []
y = []
for key, value in label_dict.items():
    data_i = classes[value]
    for data in data_i:
      X.append(data)
      y.append(one_hot(key))

X = np.array(X)
y = np.array(y)

In [152]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [153]:
# Convert data to torch
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

In [154]:
# Model Hyperparameters
input_size = 3
hidden_sizes = 32
output_size = len(categories)
n_layers = 4
dropout = 0.1
weight_decay = 0.0
epochs = 20
n_chunks = 200
learning_rate = 0.01
optimizer = 'SGD'
bidirectional= True

# Define model
model = VanillaRNN(input_size, output_size, hidden_sizes, n_layers, dropout, bidirectional)

In [155]:
# Train model
loss = train(model, X_train, y_train, epochs=epochs, learning_rate = learning_rate, weight_decay = weight_decay, n_chunks = n_chunks, optimizer = optimizer)

Training model with epochs = 20, learning rate = 0.01

Epoch: 1/20...  Loss: 418.0599
Epoch: 2/20...  Loss: 379.1228
Epoch: 3/20...  Loss: 342.6028
Epoch: 4/20...  Loss: 300.9908
Epoch: 5/20...  Loss: 260.3591
Epoch: 6/20...  Loss: 239.5989
Epoch: 7/20...  Loss: 224.8894
Epoch: 8/20...  Loss: 213.3865
Epoch: 9/20...  Loss: 203.3009
Epoch: 10/20...  Loss: 194.1901
Epoch: 11/20...  Loss: 184.9012
Epoch: 12/20...  Loss: 177.2083
Epoch: 13/20...  Loss: 171.6928
Epoch: 14/20...  Loss: 167.0449
Epoch: 15/20...  Loss: 163.3282
Epoch: 16/20...  Loss: 160.6098
Epoch: 17/20...  Loss: 155.3722
Epoch: 18/20...  Loss: 151.7895
Epoch: 19/20...  Loss: 148.3414
Epoch: 20/20...  Loss: 145.9939


In [156]:
# get accuracy
accuracy_train, accuracy_test = evaluate_model(model, X_train, y_train, X_test, y_test)

Accuracy score for train set is 0.7495029821073559 

Accuracy score for test set is 0.7438983986747654 



In [157]:
# Model Hyperparameters
input_size = 3
hidden_sizes = 32
output_size = len(categories)
n_layers = 4
dropout = 0.0
weight_decay = 0.0
epochs = 20
n_chunks = 200
learning_rate = 0.01
optimizer = 'SGD'
bidirectional = True

# Define model
model = VanillaRNNWithCNN(input_size, output_size, hidden_sizes, n_layers, dropout, bidirectional)

In [158]:
# Train model
loss = train(model, X_train, y_train, epochs=epochs, learning_rate = learning_rate, weight_decay = weight_decay, n_chunks = n_chunks, optimizer = optimizer)

Training model with epochs = 20, learning rate = 0.01

Epoch: 1/20...  Loss: 410.0557
Epoch: 2/20...  Loss: 342.0093
Epoch: 3/20...  Loss: 293.6451
Epoch: 4/20...  Loss: 263.2213
Epoch: 5/20...  Loss: 240.8271
Epoch: 6/20...  Loss: 223.5692
Epoch: 7/20...  Loss: 208.3804
Epoch: 8/20...  Loss: 194.0322
Epoch: 9/20...  Loss: 173.3156
Epoch: 10/20...  Loss: 155.9448
Epoch: 11/20...  Loss: 146.1771
Epoch: 12/20...  Loss: 137.2430
Epoch: 13/20...  Loss: 130.6573
Epoch: 14/20...  Loss: 125.3805
Epoch: 15/20...  Loss: 120.8006
Epoch: 16/20...  Loss: 116.2722
Epoch: 17/20...  Loss: 112.7427
Epoch: 18/20...  Loss: 109.5250
Epoch: 19/20...  Loss: 106.4043
Epoch: 20/20...  Loss: 103.3892


In [159]:
# get accuracy
accuracy_train, accuracy_test = evaluate_model(model, X_train, y_train, X_test, y_test)

Accuracy score for train set is 0.8311795891318754 

Accuracy score for test set is 0.8230811706239647 



In [160]:
# Model Hyperparameters
input_size = 3
hidden_sizes = 32
output_size = len(categories)
n_layers = 3
dropout = 0.0
weight_decay = 0.0
epochs = 20
n_chunks = 100
learning_rate = 0.1
optimizer = 'SGD'
bidirectional = True

# Define model
model = VanillaLSTM(input_size, output_size, hidden_sizes, n_layers, dropout, bidirectional)

In [161]:
# Train model
loss = train(model, X_train, y_train, epochs=epochs, learning_rate = learning_rate, weight_decay = weight_decay, n_chunks = n_chunks, optimizer = optimizer)

Training model with epochs = 20, learning rate = 0.1

Epoch: 1/20...  Loss: 170.6856
Epoch: 2/20...  Loss: 130.0653
Epoch: 3/20...  Loss: 105.9991
Epoch: 4/20...  Loss: 86.3209
Epoch: 5/20...  Loss: 75.8939
Epoch: 6/20...  Loss: 69.7249
Epoch: 7/20...  Loss: 64.6209
Epoch: 8/20...  Loss: 60.9655
Epoch: 9/20...  Loss: 56.2669
Epoch: 10/20...  Loss: 53.0282
Epoch: 11/20...  Loss: 49.8089
Epoch: 12/20...  Loss: 47.4998
Epoch: 13/20...  Loss: 45.5068
Epoch: 14/20...  Loss: 44.0345
Epoch: 15/20...  Loss: 41.9154
Epoch: 16/20...  Loss: 40.4857
Epoch: 17/20...  Loss: 38.6409
Epoch: 18/20...  Loss: 38.2479
Epoch: 19/20...  Loss: 36.3907
Epoch: 20/20...  Loss: 35.8636


In [162]:
# get accuracy
accuracy_train, accuracy_test = evaluate_model(model, X_train, y_train, X_test, y_test)

Accuracy score for train set is 0.8766291142036668 

Accuracy score for test set is 0.8685808945334069 



In [163]:
# Model Hyperparameters
input_size = 3
hidden_sizes = 32
output_size = len(categories)
n_layers = 4
dropout = 0.05
weight_decay = 0.0
epochs = 10
n_chunks = 200
learning_rate = 0.1
optimizer = 'SGD'
bidirectional = True

# Define model
model = VanillaLSTMWithCNN(input_size, output_size, hidden_sizes, n_layers, dropout, bidirectional)

In [164]:
# Train model
loss = train(model, X_train, y_train, epochs=epochs, learning_rate = learning_rate, weight_decay = weight_decay, n_chunks = n_chunks, optimizer = optimizer)

Training model with epochs = 10, learning rate = 0.1

Epoch: 1/10...  Loss: 264.0151
Epoch: 2/10...  Loss: 158.5255
Epoch: 3/10...  Loss: 124.6494
Epoch: 4/10...  Loss: 105.8081
Epoch: 5/10...  Loss: 91.6933
Epoch: 6/10...  Loss: 81.3721
Epoch: 7/10...  Loss: 74.9141
Epoch: 8/10...  Loss: 70.0046
Epoch: 9/10...  Loss: 66.2039
Epoch: 10/10...  Loss: 63.0440


In [165]:
# get accuracy
accuracy_train, accuracy_test = evaluate_model(model, X_train, y_train, X_test, y_test)

Accuracy score for train set is 0.8926993593991606 

Accuracy score for test set is 0.8779679734953064 



In [166]:
# Model Hyperparameters
input_size = 3
hidden_sizes = 32
output_size = len(categories)
n_layers = 3
dropout = 0.0
weight_decay = 0.0
epochs = 20
n_chunks = 100
learning_rate = 0.1
optimizer = 'SGD'
bidirectional = True

# Define model
model = VanillaGRU(input_size, output_size, hidden_sizes, n_layers, dropout, bidirectional)

In [167]:
# Train model
loss = train(model, X_train, y_train, epochs=epochs, learning_rate = learning_rate, weight_decay = weight_decay, n_chunks = n_chunks, optimizer = optimizer)

Training model with epochs = 20, learning rate = 0.1

Epoch: 1/20...  Loss: 167.9943
Epoch: 2/20...  Loss: 127.0240
Epoch: 3/20...  Loss: 107.5064
Epoch: 4/20...  Loss: 90.1924
Epoch: 5/20...  Loss: 75.5425
Epoch: 6/20...  Loss: 65.4412
Epoch: 7/20...  Loss: 58.8963
Epoch: 8/20...  Loss: 53.4405
Epoch: 9/20...  Loss: 49.5826
Epoch: 10/20...  Loss: 46.0944
Epoch: 11/20...  Loss: 43.0165
Epoch: 12/20...  Loss: 40.9014
Epoch: 13/20...  Loss: 38.8624
Epoch: 14/20...  Loss: 37.2167
Epoch: 15/20...  Loss: 35.9022
Epoch: 16/20...  Loss: 34.6183
Epoch: 17/20...  Loss: 33.4115
Epoch: 18/20...  Loss: 32.4174
Epoch: 19/20...  Loss: 31.4944
Epoch: 20/20...  Loss: 30.7709


In [168]:
# get accuracy
accuracy_train, accuracy_test = evaluate_model(model, X_train, y_train, X_test, y_test)

Accuracy score for train set is 0.8924508504528386 

Accuracy score for test set is 0.8742131419105467 



In [169]:
# Model Hyperparameters
input_size = 3
hidden_sizes = 32
output_size = len(categories)
n_layers = 3
dropout = 0.0
weight_decay = 0.0
epochs = 20
n_chunks = 100
learning_rate = 0.1
optimizer = 'SGD'
bidirectional = True

# Define model
model = VanillaGRUWithCNN(input_size, output_size, hidden_sizes, n_layers, dropout, bidirectional)

In [170]:
# Train model
loss = train(model, X_train, y_train, epochs=epochs, learning_rate = learning_rate, weight_decay = weight_decay, n_chunks = n_chunks, optimizer = optimizer)

Training model with epochs = 20, learning rate = 0.1

Epoch: 1/20...  Loss: 152.2372
Epoch: 2/20...  Loss: 99.7676
Epoch: 3/20...  Loss: 73.7976
Epoch: 4/20...  Loss: 60.9642
Epoch: 5/20...  Loss: 53.1641
Epoch: 6/20...  Loss: 47.5618
Epoch: 7/20...  Loss: 43.0744
Epoch: 8/20...  Loss: 39.5680
Epoch: 9/20...  Loss: 36.7835
Epoch: 10/20...  Loss: 34.5181
Epoch: 11/20...  Loss: 32.6944
Epoch: 12/20...  Loss: 31.1433
Epoch: 13/20...  Loss: 29.8523
Epoch: 14/20...  Loss: 28.7488
Epoch: 15/20...  Loss: 27.7439
Epoch: 16/20...  Loss: 26.8560
Epoch: 17/20...  Loss: 26.1235
Epoch: 18/20...  Loss: 25.4106
Epoch: 19/20...  Loss: 24.7633
Epoch: 20/20...  Loss: 24.1525


In [171]:
# get accuracy
accuracy_train, accuracy_test = evaluate_model(model, X_train, y_train, X_test, y_test)

Accuracy score for train set is 0.9181577203445991 

Accuracy score for test set is 0.8965212589729431 



In [172]:
# drawing = [[[121,107,45,17,1,0,4,21,58,118,173,197,209,224,244,254,254,209,164,124],[47,57,56,75,93,114,123,140,162,187,196,187,177,164,136,115,101,83,71,43]],[[123,126],[43,0]]]
# draw(drawing)